In [1]:
# %matplotlib inline
import os
import glob
from pathlib import Path

import matplotlib as mpl
mpl.use('pgf')
from matplotlib import pyplot as plt
from matplotlib import gridspec

import numpy as np
import nibabel as nb
import seaborn as sn
from nilearn.image import concat_imgs

sn.set_style("whitegrid", {
    'ytick.major.size': 5,
    'xtick.major.size': 5,
})
sn.set_context("notebook", font_scale=1)

pgf_with_custom_preamble = {
    'ytick.major.size': 0,
    'xtick.major.size': 0,
    'font.sans-serif': ['HelveticaLTStd-Light'],
    'font.family': 'sans-serif', # use serif/main font for text elements
    'text.usetex': False,    # use inline math for ticks
}
mpl.rcParams.update(pgf_with_custom_preamble)


pgf_with_custom_preamble = {
#     'font.sans-serif': ['Helvetica Light'],
#     'font.family': 'sans-serif', # use serif/main font for text elements
    'text.usetex': True,    # use inline math for ticks
    'pgf.rcfonts': False,   # don't setup fonts from rc parameters
    'pgf.texsystem': 'xelatex',
    'verbose.level': 'debug-annoying',
    "pgf.preamble": [
#         r'\renewcommand{\sfdefault}{phv}',
#         r'\usepackage[scaled=.92]{helvet}',
        r"""\usepackage{fontspec}
\setsansfont{HelveticaLTStd-Light}[
Extension=.otf,
BoldFont=HelveticaLTStd-Bold,
ItalicFont=HelveticaLTStd-LightObl,
BoldItalicFont=HelveticaLTStd-BoldObl,
]
\setmainfont{HelveticaLTStd-Light}[
Extension=.otf,
BoldFont=HelveticaLTStd-Bold,
ItalicFont=HelveticaLTStd-LightObl,
BoldItalicFont=HelveticaLTStd-BoldObl,
]
\setmonofont{Inconsolata-dz}
""",
        r'\renewcommand\familydefault{\sfdefault}',
#         r'\setsansfont[Extension=.otf]{Helvetica-LightOblique}',
#         r'\setmainfont[Extension=.ttf]{DejaVuSansCondensed}',
#         r'\setmainfont[Extension=.otf]{FiraSans-Light}',
#         r'\setsansfont[Extension=.otf]{FiraSans-Light}',
    ]
}
mpl.rcParams.update(pgf_with_custom_preamble)

In [2]:
from nilearn import plotting

In [3]:
derivs_home = os.path.expanduser('~/tmp/sherlock')
work_dir = os.path.expanduser('~/tmp/sherlock-work')
fprep_home = os.path.join(derivs_home, 'fmriprep-1.0.3', 'fmriprep')
fprep_pattern = os.path.join(fprep_home, 'sub-*', 'func',
                             'sub-*_task-stopsignal_bold_space-MNI152NLin2009cAsym_preproc.nii.gz')

In [4]:
if not os.path.isfile(os.path.join(work_dir, 'fmriprep_std.nii.gz')):
    fprep_files = glob.glob(os.path.join(fprep_home, 'sub-*', 'func',
                            'sub-*_task-stopsignal_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'))
    all_mus = []
    for f in fprep_files:
        nii = nb.load(f)
        std = nii.get_data().mean(3)
        newnii = nb.Nifti1Image(std, nii.affine, nii.header)
        newnii.set_data_dtype(float)
        all_mus.append(newnii)

    meannii = concat_imgs(all_mus, auto_resample=True)
    meannii.to_filename(os.path.join(work_dir, 'fmriprep_means.nii.gz'))

    nb.Nifti1Image(meannii.get_data().std(3), meannii.affine, meannii.header).to_filename(
        os.path.join(work_dir, 'fmriprep_std.nii.gz'))
    

In [5]:
if not os.path.isfile(os.path.join(work_dir, 'feat_std.nii.gz')):
    feat_home = os.path.join(derivs_home, 'fslfeat_5.0.9')
    feat_files = glob.glob(os.path.join(feat_home, 'sub-*.feat', 'reg',
                           'example_func2standard.nii.gz'))

    meannii = concat_imgs([nb.load(f) for f in feat_files], auto_resample=True)
    meannii.to_filename(os.path.join(work_dir, 'feat_means.nii.gz'))

    nb.Nifti1Image(meannii.get_data().std(3), meannii.affine, meannii.header).to_filename(
        os.path.join(work_dir, 'feat_std.nii.gz'))

In [6]:
import nilearn as nl

fmriprep_std = nb.load(os.path.join(work_dir, 'fmriprep_std.nii.gz'))
feat_std = nb.load(os.path.join(work_dir, 'feat_std.nii.gz'))
bmask = nb.load(os.path.expanduser('/home/oesteban/.cache/stanford-crn/mni_icbm152_nlin_asym_09c/2mm_brainmask.nii.gz')).get_data()

newmask = nl.image.resample_to_img(os.path.expanduser('/home/oesteban/.cache/stanford-crn/mni_icbm152_nlin_asym_09c/2mm_brainmask.nii.gz'), os.path.join(work_dir, 'fmriprep_std.nii.gz'), interpolation='nearest')

In [7]:
fig = plt.gcf()
_ = fig.set_size_inches(15, 2 * 3.1)
gs = gridspec.GridSpec(2, 2, width_ratios=[6, 1], height_ratios=[1, 1],  hspace=0.0, wspace=0.0)

ax1 = plt.subplot(gs[0, :-1])

disp = plotting.plot_anat(os.path.join(work_dir, 'fmriprep_std.nii.gz'), display_mode='z',
                          cut_coords=[-15, -5, 10, 20, 40], cmap='cividis', threshold=30, vmin=50, vmax=150,
                          axes=ax1)
disp.add_contours(os.path.expanduser('/home/oesteban/.cache/stanford-crn/mni_icbm152_nlin_asym_09c/1mm_tpm_csf.nii.gz'), colors=['k'], levels=[0.8])
disp.add_contours(os.path.expanduser('/home/oesteban/.cache/stanford-crn/mni_icbm152_nlin_asym_09c/1mm_tpm_wm.nii.gz'), colors=['w'], levels=[0.8], linewidths=[1], alpha=0.7)
disp.add_contours(os.path.expanduser('/home/oesteban/.cache/stanford-crn/mni_icbm152_nlin_asym_09c/1mm_brainmask.nii.gz'), colors=['k'], levels=[0.8], linewidths=[3], alpha=.7)

ax2 = plt.subplot(gs[1, :-1])
disp = plotting.plot_anat(os.path.join(work_dir, 'feat_std.nii.gz'), display_mode='z',
                          cut_coords=[-15, -5, 10, 20, 40], cmap='cividis', threshold=30, vmin=50, vmax=150,
                          axes=ax2)
disp.add_contours(os.path.expanduser('/home/oesteban/.cache/stanford-crn/mni_icbm152_nlin_asym_09c/1mm_tpm_csf.nii.gz'), colors=['k'], levels=[0.8])
disp.add_contours(os.path.expanduser('/home/oesteban/.cache/stanford-crn/mni_icbm152_nlin_asym_09c/1mm_tpm_wm.nii.gz'), colors=['w'], levels=[0.8], linewidths=[1], alpha=0.7)
disp.add_contours(os.path.expanduser('/home/oesteban/.cache/stanford-crn/mni_icbm152_nlin_asym_09c/1mm_brainmask.nii.gz'), colors=['k'], levels=[0.8], linewidths=[3], alpha=.7)

ax1.annotate(
    'fMRIPrep',
    xy=(0., .5), xycoords='axes fraction', xytext=(-24, .0),
    textcoords='offset points', va='center', color='k', size=24,
    rotation=90)

ax2.annotate(
    r'\texttt{feat}',
    xy=(0., .5), xycoords='axes fraction', xytext=(-24, .0),
    textcoords='offset points', va='center', color='k', size=24,
    rotation=90)


inner_grid = gridspec.GridSpecFromSubplotSpec(1, 2, width_ratios=[1, 15],
                                              subplot_spec=gs[:, 1], wspace=0.01)

ax3 = fig.add_subplot(inner_grid[0])
gradient = np.hstack((np.zeros((50,)), np.linspace(0, 1, 120), np.ones((130,))))[::-1]
gradient = np.vstack((gradient, gradient))
ax3.imshow(gradient.T, aspect='auto', cmap=plt.get_cmap('cividis'))
ax3.xaxis.set_ticklabels([])
ax3.xaxis.set_ticks([])
ax3.yaxis.set_ticklabels([])
ax3.yaxis.set_ticks([])

ax4 = fig.add_subplot(inner_grid[1])
sn.distplot(fmriprep_std.get_data()[newmask.get_data() > 0], label='fMRIPrep', vertical=True, ax=ax4)
sn.distplot(feat_std.get_data()[bmask == 1], label=r'\texttt{feat}', vertical=True, color='darkorange', ax=ax4)

plt.gca().set_ylim((0, 300))
plt.legend(prop={'size': 20}, edgecolor='none')

ax4.xaxis.set_ticklabels([])
ax4.xaxis.set_ticks([])
ax4.yaxis.set_ticklabels([])
ax4.yaxis.set_ticks([])

plt.axis('off')
ax3.axis('off')
ax4.axis('off')

plt.savefig(str(Path.home() / 'Dropbox' / 'My Publications' / '2017-FMRIPREP' / 'figures' / 'fmriprep-feat-std.pdf'),
            format='pdf', bbox_inches='tight', pad_inches=0.2, dpi=300)

/home/oesteban/.anaconda3/lib/python3.6/site-packages/matplotlib/contour.py:1173: UserWarning: No contour levels were found within the data range.
  warnings.warn("No contour levels were found"
/home/oesteban/.anaconda3/lib/python3.6/site-packages/matplotlib/contour.py:1173: UserWarning: No contour levels were found within the data range.
  warnings.warn("No contour levels were found"
/home/oesteban/.anaconda3/lib/python3.6/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


In [ ]:
for i, coords in enumerate([-5, 10, 20]):
    disp = plotting.plot_anat(os.path.join(work_dir, 'fmriprep_std.nii.gz'), display_mode='z', cut_coords=[coords], cmap='viridis', threshold=30, vmin=50, vmax=170)
    f = plt.gcf().set_size_inches(10, 20)
    disp.add_contours(os.path.expanduser('/home/oesteban/.cache/stanford-crn/mni_icbm152_nlin_asym_09c/1mm_tpm_csf.nii.gz'), colors=['k'], levels=[0.8])
    disp.add_contours(os.path.expanduser('/home/oesteban/.cache/stanford-crn/mni_icbm152_nlin_asym_09c/1mm_tpm_wm.nii.gz'), colors=['w'], levels=[0.8], alpha=0.7)
    disp.add_contours(os.path.expanduser('/home/oesteban/.cache/stanford-crn/mni_icbm152_nlin_asym_09c/1mm_brainmask.nii.gz'), colors=['k'], levels=[0.8], linewidths=[3], alpha=.7)
    plt.savefig(os.path.join(work_dir, 'fmriprep-std-closeup%03d.svg' % i), format='svg', bbox_inches='tight')

In [ ]:
for i, coords in enumerate([-5, 10, 20]):
    disp = plotting.plot_anat(os.path.join(work_dir, 'feat_std.nii.gz'), display_mode='z', cut_coords=[coords], cmap='viridis', threshold=30, vmin=50, vmax=170)
    f = plt.gcf().set_size_inches(10, 20)
    disp.add_contours(os.path.expanduser('/home/oesteban/.cache/stanford-crn/mni_icbm152_nlin_asym_09c/1mm_tpm_csf.nii.gz'), colors=['k'], levels=[0.8])
    disp.add_contours(os.path.expanduser('/home/oesteban/.cache/stanford-crn/mni_icbm152_nlin_asym_09c/1mm_tpm_wm.nii.gz'), colors=['w'], levels=[0.8], alpha=0.7)
    disp.add_contours(os.path.expanduser('/home/oesteban/.cache/stanford-crn/mni_icbm152_nlin_asym_09c/1mm_brainmask.nii.gz'), colors=['k'], levels=[0.8], linewidths=[3], alpha=.7)
    plt.savefig(os.path.join(work_dir, 'feat-std-closeup%03d.svg' % i), format='svg', bbox_inches='tight')

In [ ]:
feat_files[137]

In [ ]:
disp = plotting.plot_glass_brain(os.path.join(work_dir, 'feat_std.nii.gz'))